# Desafio - Etapa 1 
---

Nessa etapa serão executados códigos para realização da limpeza do arquvivo `csv`. Além disso, serão definidos três análises para os dados escolhidos. Em seguida a biblioteca **boto3** será utilizada para carregar o arquivo limpo para um bucket.

In [3]:
# Importando Bibliotecas 
import pandas as pd
import boto3
from botocore.exceptions import ClientError
import os
from dotenv import load_dotenv
import io 

## Upload para Bucket e Tratamento do DataFrame
---

In [4]:
# Carregando as variáveis do arquivo .env
load_dotenv()

True

### Carregamento seguro das credenciais para conexão com a AWS
---

In [5]:
# Acessando credenciais do arquivo .env
aws_key_id = os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_key = os.getenv('AWS_SECRET_ACCESS_KEY')
aws_token =  os.getenv('AWS_SESSION_TOKEN')
aws_region = 'us-east-1'

In [6]:
# Validação para garantir que as chaves foram carregadas
if not all([aws_key_id, aws_secret_key]):
    raise ValueError("Credenciais da AWS não encontradas no arquivo .env ou no ambiente.")

In [7]:
# Inicando o clietne S3
s3_client = boto3.client('s3',
    aws_access_key_id=aws_key_id,
    aws_secret_access_key=aws_secret_key,
    aws_session_token=aws_token,
    region_name=aws_region                     
)

### Configuração do Bucket
---

In [8]:
# Definindo nomes e caminhos
nome_bucket = "bucket-desafio-2497"
# Caminho do arquivo original 
caminho_original = '../Arquivo/Dados abertos 2021_CPP_CSB_Itens incorporados ao acervo_Obras em geral.csv'
# Nomeando objetos que serão salvos no S3
s3_key_original = 'original/dados_originais_2021.csv'
s3_key_limpo = 'limpo/dados_limpos_2021.csv'

### Criação do Bucket
---

In [9]:
# Criando o bucket
if aws_region == 'us-east-1':
        s3_client.create_bucket(Bucket=nome_bucket)
else:
    location = {'LocationConstraint': aws_region}
    s3_client.create_bucket(Bucket=nome_bucket, CreateBucketConfiguration=location)

### Upload do DataFrame Original
---

In [10]:
try:
    print(f"Iniciando upload do arquivo original para '{s3_key_original}'...")
    s3_client.upload_file(caminho_original, nome_bucket, s3_key_original)
    print("Upload do arquivo original concluído com sucesso.")
except FileNotFoundError:
    print(f"ERRO: O arquivo original não foi encontrado em '{caminho_original}'.")
except ClientError as e:
    print(f"ERRO ao fazer upload do arquivo original: {e}")

Iniciando upload do arquivo original para 'original/dados_originais_2021.csv'...
Upload do arquivo original concluído com sucesso.


### Carregamento e Limpeza de Dados
---

In [11]:
df = pd.read_csv('../Arquivo/Dados abertos 2021_CPP_CSB_Itens incorporados ao acervo_Obras em geral.csv', 
    delimiter=';',
    encoding='latin-1'
) 
df

,Título,N. tombo,ISBN,Edição,Volume,Tipo de material,Forma de aquisição,Data do registro
0,"""Das trevas da ignorância para alcançar a luz ...",1.612.826,9.788593e+12,NaN,NaN,Livro,DL,29/12/2021
1,"""O pilão de pilar lembranças""",1.612.465,9.788583e+12,NaN,NaN,Livro,DL,06/12/2021
2,"""Somos todos COS!""",1.612.663,9.788533e+12,1. ed.,NaN,Livro,DL,15/12/2021
3,#Selfies,1.612.601,9.788521e+12,NaN,NaN,Livro,DL,09/12/2021
4,[Negros carregadores com seus cestos],1.250.053,NaN,NaN,NaN,Original de arte,NaN,14/03/2021
...,...,...,...,...,...,...,...,...
650,Vivi,1.612.338,9.788555e+12,NaN,NaN,Livro,DL,03/12/2021
651,Walter Benjamin,1.612.320,9.788531e+12,"1. ed., 2. reimpr.",NaN,Livro,DL,03/12/2021
652,We are all outsiders,1.612.437,9.788535e+12,NaN,NaN,Livro,DL,06/12/2021
653,"Yutiguar, o charrua",1.612.542,9.788521e+12,NaN,NaN,Livro,DL,07/12/2021


*Obs: Quando houve a tentativa de leitura do arquivo normalmente, foi retornado o erro `UnicodeDecodeError`, por isso o `encoding`foi configurado para o `latin-1`. Isso se fez necessário devido aos caracteres especiais do arquivo `ç` e `ã`, por exemplo.*

- **Limpeza do DataFrame**

Devido a coluna 'Volume' ter muitos dados nulos, foi preferível removê-la. Ademais, também foi removida a coluna 'Edição', pois nenhuma análise seria realizada por meio dela.

In [12]:
# Colunas para remoção
colunas_remover = ['Volume', 'Edição']
# Removendo colunas
df_limpo = df.drop(columns=colunas_remover).copy()
df_limpo

,Título,N. tombo,ISBN,Tipo de material,Forma de aquisição,Data do registro
0,"""Das trevas da ignorância para alcançar a luz ...",1.612.826,9.788593e+12,Livro,DL,29/12/2021
1,"""O pilão de pilar lembranças""",1.612.465,9.788583e+12,Livro,DL,06/12/2021
2,"""Somos todos COS!""",1.612.663,9.788533e+12,Livro,DL,15/12/2021
3,#Selfies,1.612.601,9.788521e+12,Livro,DL,09/12/2021
4,[Negros carregadores com seus cestos],1.250.053,NaN,Original de arte,NaN,14/03/2021
...,...,...,...,...,...,...
650,Vivi,1.612.338,9.788555e+12,Livro,DL,03/12/2021
651,Walter Benjamin,1.612.320,9.788531e+12,Livro,DL,03/12/2021
652,We are all outsiders,1.612.437,9.788535e+12,Livro,DL,06/12/2021
653,"Yutiguar, o charrua",1.612.542,9.788521e+12,Livro,DL,07/12/2021


Na coluna 'Forma de aquisição' há muitos valores nulos que serão corrigidos para o valor 'Não especificada'.

In [13]:
# Preenchendo valores nulos 
df_limpo['Forma de aquisição'] = df_limpo['Forma de aquisição'].fillna('Não especificada')
df_limpo

,Título,N. tombo,ISBN,Tipo de material,Forma de aquisição,Data do registro
0,"""Das trevas da ignorância para alcançar a luz ...",1.612.826,9.788593e+12,Livro,DL,29/12/2021
1,"""O pilão de pilar lembranças""",1.612.465,9.788583e+12,Livro,DL,06/12/2021
2,"""Somos todos COS!""",1.612.663,9.788533e+12,Livro,DL,15/12/2021
3,#Selfies,1.612.601,9.788521e+12,Livro,DL,09/12/2021
4,[Negros carregadores com seus cestos],1.250.053,NaN,Original de arte,Não especificada,14/03/2021
...,...,...,...,...,...,...
650,Vivi,1.612.338,9.788555e+12,Livro,DL,03/12/2021
651,Walter Benjamin,1.612.320,9.788531e+12,Livro,DL,03/12/2021
652,We are all outsiders,1.612.437,9.788535e+12,Livro,DL,06/12/2021
653,"Yutiguar, o charrua",1.612.542,9.788521e+12,Livro,DL,07/12/2021


Foi necessário a normalização dos valores das colunas 'Tipo de Material' e 'Forma de aquisição' apenas para garantir segurança as análises que serão realizadas.

In [14]:
# Padronizando valores das colunas 
df_limpo['Tipo de material'] = df_limpo['Tipo de material'].str.strip().str.upper()
df_limpo['Forma de aquisição'] = df_limpo['Forma de aquisição'].str.strip().str.upper()

### Upload do DataFrame Limpo
---


In [15]:
# Informações sobre o DataFrame limpo
df_limpo.info()
# Verificando se o DataFrame foi limpo
if 'df_limpo' in locals():
    try:
        print(f"\nIniciando upload do arquivo limpo para '{s3_key_limpo}'...")
        # Convertendo o DataFrame para um objeto CSV em memória
        csv_buffer = io.StringIO()
        df_limpo.to_csv(csv_buffer, index=False, sep=';', encoding='utf-8')        
        # Fazendo o upload do buffer em memória para o S3
        s3_client.put_object(Bucket=nome_bucket, Key=s3_key_limpo, Body=csv_buffer.getvalue())        
        print("Upload do arquivo limpo concluído com sucesso.")
        
    except ClientError as e:
        print(f"ERRO ao fazer upload do arquivo limpo: {e}")
else:
    print("\nUpload do arquivo limpo não realizado devido a erros.")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 655 entries, 0 to 654
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Título              655 non-null    object 
 1   N. tombo            655 non-null    object 
 2   ISBN                598 non-null    float64
 3   Tipo de material    655 non-null    object 
 4   Forma de aquisição  655 non-null    object 
 5   Data do registro    655 non-null    object 
dtypes: float64(1), object(5)
memory usage: 30.8+ KB

Iniciando upload do arquivo limpo para 'limpo/dados_limpos_2021.csv'...
Upload do arquivo limpo concluído com sucesso.


## Análises que Serão Realizadas
---

Foi selecionadas as três seguintes análises para o DataFrame: 

- Análise 1: Descobrindo quais meses o acervo recebe mais itens.
- Análise 2: Descobrindo quais livros contêm a palavra "história" no título.
- Análise 3: Classificação por Semestre de Registro.